In [2]:
import numpy as np
import matplotlib as plt

In [92]:
# n = 3
# A = np.array((1, 2, 3, 2, 4, 5, 3, 5, 6)).reshape(n,n) # n = 3
n = 2
A = np.array((2, 1, 1, 4,)).reshape(n, n)

A

array([[2, 1],
       [1, 4]])

In [93]:
b = np.array((2, 3))

b

array([2, 3])

In [97]:
m = 3
k = n*(n+1)//2
z = np.array((3, 2, 1, 1, 0, -2)).reshape(m, n)

z

array([[ 3,  2],
       [ 1,  1],
       [ 0, -2]])

In [98]:
# Residual for single point
def r(A, v, zi, wi):
    x = zi.T @ A @zi + b.T @ zi
    return np.maximum(wi * x, 0)

# Residual vector for all points
def R(A, b, z, w):
    R = np.zeros_like(w)
    for i in range(R.size):
        R[i] = r(A, b, z[i], w[i])
    return R

#Objective function
def f(R):
    return np.sum(R**2)

array([[ 8, 11],
       [ 3,  5],
       [-2, -8]])

In [1]:
# Insert coefficients in (nxn) matrix A, given (m+k) vector x
def update_A(A, x):
    n = A.shape[0]
    a = x[:n*(n+1)//2]
    for j in range(n):
        start = j*n - j*(j+1)//n
        end = start + n-j
        A[j, j:] = x[start:end]
    return A

# Insert coefficients in (n) centering vector given (n+k) vector x
def update_c(c, x):
    c = x[-n:]
    return c

# Get the first (k) coefficients of (n+k) vector x, given an (nxn) matrix A
def extract_a(A):
    n = A.shape[0]
    k = n*(n+1)//2
    a = np.zeros(k)
    end = 0
    
    for j in range(n):
        start = end
        end = start + n-j
        a[start:end] = A[j, j:]
        
    return a

In [3]:
# return partial derivatives of residual i, wrt. the matrix coefficients
def r_delA(A, b, zi):
    n = b.size
    rdA = 2 * (v * (v * np.ones_like(A)).T)
    rdA = rdA - 1/2 * np.identity(n) * rdA
    return extract_a(rdA)

# Return vector of all partial derivatives of residual i
def del_r(A, b, zi):
    n = np.size
    k = n*(n+1)//2
    dr = np.zeros(k+n)
    dr[:k] = r_delA(A, c, zi)
    dr[k:] = zi

#Jacobian of residual vector
def jacobian_R(A, c, z):
    m = z.shape[0]
    n = z.shape[1]
    k = n*(n+1)//2
    R = np.zeros((m, k+n))
    for i in range(m):
        R[i] = del_r(A, c, z[i])
    return R
    

In [96]:
h = lambda A, b, z: z.T @ A @ z + b @ z - 1
h(A, b, z[0])

57

In [100]:
def r_delA(A, c, z)
    